In [ ]:
#Raw sequence preprocessing

In [ ]:
#Library loading
library(dada2)

In [ ]:
path = "/Users/AJaser/Documents/Buffel/fastq_buffel"
list.files(path)
#Getting matched lists of the forward and reverse fastq files
fnFs = sort(list.files(path, pattern="_R1.fastq", full.names = TRUE))
#fnFs
fnRs = sort(list.files(path, pattern="_R2.fastq", full.names = TRUE))
#fnRs
# Extract sample names
sample.names = sapply(strsplit(basename(fnFs), "_"), `[`, 1)
sample.names

In [ ]:
#Visualizing the quality profiles of the forward and reverse reads
plotQualityProfile(fnFs[1:2])
plotQualityProfile(fnRs[1:2])

In [ ]:
#Trimming sequences
fout = filterAndTrim(fnFs, filtFs, trimLeft=17, truncLen=250,
              maxN=0, maxEE=6, truncQ=2, rm.phix=TRUE,
              compress=TRUE, multithread=TRUE) 
fout

In [ ]:
#Plotting error rates using a parametric error model
ferrF = learnErrors(filtFs, multithread=TRUE)
plotErrors(ferrF, nominalQ=TRUE)

In [ ]:
#Applying the core sample inference algorithm to the filtered and trimmed sequence data
fdadaFs = dada(filtFs, err=ferrF, multithread=TRUE)

In [ ]:
#Creating sequence table using the filtered Fw reads file
fseqtab = makeSequenceTable(fdadaFs)
dim(fseqtab)
row.names(fseqtab)
# Inspect distribution of sequence lengths
table(nchar(getSequences(fseqtab)))

In [ ]:
#Removing chimeras
fseqtab.nochim = removeBimeraDenovo(fseqtab, method="consensus", multithread=TRUE, verbose=TRUE)
dim(fseqtab.nochim)
#Frequency of chimeric sequences
sum(fseqtab.nochim)/sum(fseqtab)

In [ ]:
#Looking at the number of reads that made it through each step in the pipeline
fgetN = function(x) sum(getUniques(x))
ftrack = cbind(fout, sapply(fdadaFs, fgetN), rowSums(fseqtab.nochim))
colnames(ftrack) = c("input", "filtered", "denoisedF", "nonchim")
rownames(ftrack) = sample.names
ftrack

In [ ]:
#Assign taxonomy using SILVA database
ftaxa = assignTaxonomy(fseqtab.nochim, "/Users/AJaser/Documents/Buffel/ASV/silva_nr_v132_train_set.fa.gz", multithread=TRUE)

In [ ]:
#Assign taxonomy to species level using SILVA database
ftaxa_sp = addSpecies(ftaxa, "/Users/AJaser/Documents/Buffel/ASV/silva_species_assignment_v132.fa.gz")

In [ ]:
#Inspecting taxonomic assignments at species level
ftaxa.print = ftaxa_sp
# Removing sequence rownames for display only
rownames(ftaxa.print) = NULL
ftaxa.print

In [ ]:
theme_set(theme_bw())
# Giving our seq headers more manageable names (ASV_1, ASV_2...)
fasv_seqs = colnames(fseqtab.nochim)
fasv_headers = vector(dim(fseqtab.nochim)[2], mode="character")

In [ ]:
for (i in 1:dim(fseqtab.nochim)[2]) {
  fasv_headers[i] = paste(">ASV", i, sep="_")
}

In [ ]:
#Making and writing out a fasta of our final ASV seqs:
fasv_fasta = c(rbind(fasv_headers, fasv_seqs))
write(fasv_fasta, "ASVs_Fw.fa")

In [ ]:
#Count table:
fasv_tab = t(fseqtab.nochim)
row.names(fasv_tab) = sub(">", "", fasv_headers)
write.table(fasv_tab, "ASVs_Fw_counts.tsv", sep="\t", quote=F, col.names=NA)

In [ ]:
#Tax table:
fasv_tax = ftaxa_sp
row.names(fasv_tax) = sub(">", "", fasv_headers)
write.table(fasv_tax, "ASVs_Fw_taxonomy.tsv", sep="\t", quote=F, col.names=NA)

In [ ]:
#Creating a phylogenetic tree
qsub -pe completenode 4 -N arbolote -b y -j y -cwd -V " FastTreeMP -nt -gtr ASVs_Fw_aligned.fasta >tree_ASV_Fw.nwk"